# Epipolar_Geometry.ipynb  
By: Shawn Vosburg (260743167)  
Due: November 5th, 2018  
Course: ECSE415, McGill University  

In [ ]:
import sys
import re
from struct import *
import cv2
import numpy as np
from matplotlib import pyplot as plt

def read_pfm(file):
    # Adopted from https://stackoverflow.com/questions/37073108/how-to-read-pfm-files-provided-with-middlebury-dataset?rq=1
    with open(file, "rb") as f:
        # Line 1: PF=>RGB (3 channels), Pf=>Greyscale (1 channel)
        type = f.readline().decode('latin-1')
        if "PF" in type:
            channels = 3
        elif "Pf" in type:
            channels = 1
        else:
            sys.exit(1)
        # Line 2: width height
        line = f.readline().decode('latin-1')
        width, height = re.findall('\d+', line)
        width = int(width)
        height = int(height)

        # Line 3: +ve number means big endian, negative means little endian
        line = f.readline().decode('latin-1')
        BigEndian = True
        if "-" in line:
            BigEndian = False
        # Slurp all binary data
        samples = width * height * channels;
        buffer = f.read(samples * 4)
        # Unpack floats with appropriate endianness
        if BigEndian:
            fmt = ">"
        else:
            fmt = "<"
        fmt = fmt + str(samples) + "f"
        img = unpack(fmt, buffer)
        img = np.flipud(np.array(img).reshape(height, width))
    return img


#obtain images and depth map
img0 = cv2.imread('im0.png');
img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
img1 = cv2.imread('im1.png');
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
depthTruth = read_pfm('./disp0.pfm')
depthTruth[depthTruth==np.inf] = 0

# resize images for fast processing
img0 = cv2.resize(img0, None, fx=0.3, fy=0.3)
img1 = cv2.resize(img1, None, fx=0.3, fy=0.3)
depthTruth = cv2.resize(depthTruth, None, fx=0.3, fy=0.3)


plt.figure(figsize=(20,20))
plt.subplot(131)
plt.imshow(img0)
plt.title("Input Image #1"), plt.xticks([]), plt.yticks([])
plt.subplot(132)
plt.imshow(img1)
plt.title("Input Image #2"), plt.xticks([]), plt.yticks([])
plt.subplot(133)
plt.imshow(depthTruth,cmap = "gray")
plt.title("Depth Map (Truth)"), plt.xticks([]), plt.yticks([])
plt.show()